In [ ]:
import torch
import torch.nn.functional as F

from torch import Tensor
from transformers import AutoTokenizer, AutoModel

In [ ]:
# _MODEL_PATH = "/mnt/nvme/MODELS/EMBEDINGS/e5-mistral-7b-instruct/"
_MODEL_PATH = "/mnt/nvme/MODELS/EMBEDINGS/UAE-Large-V1/"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(_MODEL_PATH, device="cuda:1")
model = AutoModel.from_pretrained(
    _MODEL_PATH,
    device_map="cuda:1",
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True
)

In [ ]:
# def last_token_pool(
#     last_hidden_states: Tensor,
#     attention_mask: Tensor
#     ) -> Tensor:
    
#     left_padding = (attention_mask[:, -1].sum() == attention_mask.shape[0])
#     if left_padding:
#         return last_hidden_states[:, -1]
#     else:
#         sequence_lengths = attention_mask.sum(dim=1) - 1
#         batch_size = last_hidden_states.shape[0]
#         return last_hidden_states[torch.arange(batch_size, device=last_hidden_states.device), sequence_lengths]


In [ ]:
# def get_detailed_instruct(task_description: str, query: str) -> str:
#     return f'Instruct: {task_description}\nQuery: {query}'


In [ ]:
# Each query must come with a one-sentence instruction that describes the task
# task = 'Given a web search query, retrieve relevant passages that answer the query'
# input_texts = [
#     get_detailed_instruct(task, 'how much protein should a female eat'),
    
#     get_detailed_instruct(task, 'summit define'),
    
#     "As a general guideline, the CDC's average requirement of protein for women ages 19 to 70 is 46 grams per day. But, as you can see from this chart, you'll need to increase that if you're expecting or training for a marathon. Check out the chart below to see how much protein you should be eating each day.",
    
#     "Definition of summit for English Language Learners. : 1  the highest point of a mountain : the top of a mountain. : 2  the highest level. : 3  a meeting or series of meetings between the leaders of two or more governments."
# ]

In [ ]:
# max_length = 4096
# Tokenize the input texts
# batch_dict = tokenizer(input_texts, max_length=max_length - 1, return_attention_mask=False, padding=False, truncation=True)
# batch_dict = tokenizer(input_texts, return_attention_mask=False, padding=False, truncation=True)
# batch_dict

In [ ]:
# append eos_token_id to every input_ids
# batch_dict['input_ids'] = [input_ids + [tokenizer.eos_token_id] for input_ids in batch_dict['input_ids']]
# batch_dict = tokenizer.pad(batch_dict, padding=True, return_attention_mask=True, return_tensors='pt')
# batch_dict = {k: v.to(model.device) for k, v in batch_dict.items()}

In [ ]:
# outputs = model(**batch_dict)

In [ ]:
# embeddings = last_token_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
# embeddings.shape

In [ ]:
# normalize embeddings
# embeddings = F.normalize(embeddings, p=2, dim=1)
# scores = (embeddings[:2] @ embeddings[2:].T) * 100
# print(scores.tolist())

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cosine

In [ ]:
text1="""
USER: Hello Jarvis, how much would it cost to install new carpeting on my house's stairs? The steps are three feet broad and ten feet long. I would want to utilise a $5 per square foot carpet. 
ASSISTANT: Alright, Mr. I'll figure that out for you. <calculator> 3 * 10 30 <calculator> 30 * 5 150 </calculator>There would be a $150 overall cost.</s> Do you need my assistance with anything else right now?
"""

text2="""
USER: Hello Jarvis, how much would it cost to install new carpeting on my house's stairs? The steps are three feet broad and ten feet long. I would want to utilize a $5 per square foot carpet. 
ASSISTANT: Alright, Mr. I'll figure that out for you. <calculator> 3 * 10 30 <calculator> 30 * 5 150 </calculator>There would be a $150 overall cost.</s> Do you need my assistance with anything else right now?
"""
# text2 = input_texts[-1]

In [ ]:
text1 = tokenizer(text1, return_tensors="pt", padding=True, truncation=True)
text1 = {k: v.to(model.device) for k, v in text1.items()}
text2 = tokenizer(text2, return_tensors="pt", padding=True, truncation=True)
text2 = {k: v.to(model.device) for k, v in text2.items()}

In [ ]:
text1['input_ids'].shape

In [ ]:
# text1_embeddings = last_token_pool(model(**text1).last_hidden_state, text1['attention_mask'])
# text2_embeddings = last_token_pool(model(**text2).last_hidden_state, text2['attention_mask'])

text1_embeddings = model(**text1).last_hidden_state[:, -1]
text2_embeddings = model(**text2).last_hidden_state[:, -1]

In [ ]:
text1_embeddings.shape, text2_embeddings.shape

In [ ]:
cosine_similarity(text1_embeddings.cpu().detach().numpy(), text2_embeddings.cpu().detach().numpy())

In [ ]:
cosine(text1_embeddings.squeeze().cpu().detach().numpy(), text2_embeddings.squeeze().cpu().detach().numpy())

In [ ]:
text1_embeddings.cpu().detach().numpy()

In [ ]:
text2_embeddings.cpu().detach().numpy()